In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357271 sha256=cd899c3904e0437379131b36fe6982f9644936af3731f04e15cdf28006d2ade6
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


SISTEMA DE RECOMENDACION


In [ ]:
import pandas as pd

from surprise import Dataset, Reader
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate, train_test_split

In [ ]:
dfAnimeDetails_final=pd.read_csv('/content/dfAnimeDetails_final.csv', encoding='utf-8', engine='python')
dfAnimeDetails_final['Genres'] = dfAnimeDetails_final['Genres'].apply(lambda x: x.split("|"))
dfAnimeDetails_final.head()

,anime_id,Name,Score,Genres,Synopsis,Type,Rating,Popularity,Favorites
0,1,Cowboy Bebop,8.75,"[Action, Award Winning, Sci-Fi]","Crime is timeless. By the year 2071, humanity ...",TV,R - 17+ (violence & profanity),43,78525
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"[Action, Sci-Fi]","Another day, another bounty—such is the life o...",Movie,R - 17+ (violence & profanity),602,1448
2,6,Trigun,8.22,"[Action, Adventure, Sci-Fi]","Vash the Stampede is the man with a $$60,000,0...",TV,PG-13 - Teens 13 or older,246,15035
3,7,Witch Hunter Robin,7.25,"[Action, Drama, Mystery, Supernatural]",Robin Sena is a powerful craft user drafted in...,TV,PG-13 - Teens 13 or older,1795,613
4,8,Bouken Ou Beet,6.94,"[Adventure, Fantasy, Supernatural]",It is the dark century and the people are suff...,TV,PG - Children,5126,14


In [ ]:
genres = set(g for G in dfAnimeDetails_final['Genres'] for g in G)

for g in genres:
    dfAnimeDetails_final[g] = dfAnimeDetails_final['Genres'].transform(lambda x: int(g in x))

anime_genres = dfAnimeDetails_final.drop(columns=['anime_id', 'Name','Genres'])
anime_genres.head()

<ipython-input-47-14bd1f4ff6cd>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfAnimeDetails_final[g] = dfAnimeDetails_final['Genres'].transform(lambda x: int(g in x)) # Changed 'genres' to 'Genres'
<ipython-input-47-14bd1f4ff6cd>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfAnimeDetails_final[g] = dfAnimeDetails_final['Genres'].transform(lambda x: int(g in x)) # Changed 'genres' to 'Genres'
<ipython-input-47-14bd1f4ff6cd>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

,Score,Synopsis,Type,Rating,Popularity,Favorites,"Award Winning, Drama","Action, Supernatural, Hentai","Action, Adventure, Mystery, Supernatural","Comedy, Horror, Romance, Sci-Fi",...,"Action, Comedy, Drama, Sci-Fi","Adventure, Comedy, Romance, Sci-Fi","Gourmet, Sci-Fi","Action, Comedy","Sci-Fi, Suspense","Adventure, Horror, Romance, Supernatural","Action, Adventure, Comedy, Romance, Sci-Fi, Ecchi","Drama, Sci-Fi, Sports","Action, Avant Garde, Comedy, Supernatural, Ecchi","Action, Adventure, Comedy, Fantasy, Mystery"
0,8.75,"Crime is timeless. By the year 2071, humanity ...",TV,R - 17+ (violence & profanity),43,78525,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8.38,"Another day, another bounty—such is the life o...",Movie,R - 17+ (violence & profanity),602,1448,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8.22,"Vash the Stampede is the man with a $$60,000,0...",TV,PG-13 - Teens 13 or older,246,15035,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7.25,Robin Sena is a powerful craft user drafted in...,TV,PG-13 - Teens 13 or older,1795,613,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6.94,It is the dark century and the people are suff...,TV,PG - Children,5126,14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Select only numeric columns for cosine similarity calculation
numeric_anime_genres = anime_genres.select_dtypes(include=['number'])

cosine_sim = cosine_similarity(numeric_anime_genres, numeric_anime_genres)

In [ ]:
def anime_finder(Name):
    return dfAnimeDetails_final[dfAnimeDetails_final['Name'].str.contains(Name)]['Name'].tolist()

anime_idx = dict(zip(dfAnimeDetails_final['Name'], list(dfAnimeDetails_final.index)))
Name = anime_finder('Trigun')[0]
n_recommendations = 5

idx = anime_idx[Name]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:(n_recommendations+1)]
similar_animes = [i[0] for i in sim_scores]

print("Recomendaciones para {}:".format(Name))
for anime in dfAnimeDetails_final['Name'].iloc[similar_animes]:
    print("\t{}".format(anime))

Recomendaciones para Trigun:
	Perfect Blue
	Akira
	86
	Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season Part 2
	Steins;Gate 0


EVALUACION USANDO SVD


In [ ]:
import pandas as pd

from surprise import Dataset, Reader
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate, train_test_split

In [ ]:
Score = pd.read_csv('/content/dfAnimeDetails_final.csv')


reader = Reader(rating_scale=(Score.Score.min(), Score.Score.max()))

data = Dataset.load_from_df(Score[["anime_id", "Name", "Score"]], reader)


In [ ]:
from surprise import SVD

ratings_train, ratings_test = train_test_split(data, test_size=0.2)


model = SVD(n_factors=100, n_epochs=20, random_state=42)


model.fit(ratings_train)


predictions = model.test(ratings_test)

print("RMSE on test: {:.4f}".format(rmse(predictions, verbose=False)))

RMSE on test: 0.8885
